# Basics of Matplotlib
*[Matplotlib](https://matplotlib.org/3.1.1/index.html) is probably the most used Python package for quick visualizations and publication-quality figures. This notebook is not meant to cover all use cases of matplotlib since other people allready did this. It's meant to explain how matplotlib is used within the haiopy package.*
____

## 1. Anatomy of Matplotlib 
[A good tutorial](https://www.youtube.com/watch?v=6gdNUDs6QPc) to start with are the SciPy talks by Benjamin Root. The materials i.e. the jupyter notebooks for the tutorial are on [GitHub](https://github.com/matplotlib/AnatomyOfMatplotlib). [Another great tutorial](https://github.com/rougier/matplotlib-tutorial) which is not based on IPython is the one by Nicolas P. Rougier. Here you can learn about figures, subplots, axes and so on.

## 2. The `matplotlibrc` config file
Use this file to customize the default configuration of matplotlib. Just edit the settings in the `matplotlibrc`
file.

In [1]:
"Finding your matplotlibrc file"
import matplotlib
print(matplotlib.matplotlib_fname())

C:\Users\sklein\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc


### Navigation Keyboard Shortcuts
The default keys can be overwritten in the `matplotlibrc` file
    #keymap.fullscreen : f, ctrl+f       ## toggling
    #keymap.home : h, r, home            ## home or reset mnemonic
    #keymap.back : left, c, backspace, MouseButton.BACK  ## forward / backward keys
    #keymap.forward : right, v, MouseButton.FORWARD      ## for quick navigation
    #keymap.pan : p                      ## pan mnemonic
    #keymap.zoom : o                     ## zoom mnemonic
    #keymap.save : s, ctrl+s             ## saving current figure
    #keymap.help : f1                    ## display help about active tools
    #keymap.quit : ctrl+w, cmd+w, q      ## close the current figure
    #keymap.quit_all : W, cmd+W, Q       ## close all figures
    #keymap.grid : g                     ## switching on/off major grids in current axes
    #keymap.grid_minor : G               ## switching on/off minor grids in current axes
    #keymap.yscale : l                   ## toggle scaling of y-axes ('log'/'linear')
    #keymap.xscale : k, L                ## toggle scaling of x-axes ('log'/'linear')
    #keymap.all_axes : a                 ## enable all axes
    #keymap.copy : ctrl+c, cmd+c         ## Copy figure to clipboard

## 3. Backend
Matplotlib supports different outputs for several different use cases. There are two types of backends: interactive user interface backends (pygtk, wxpython, tkinter, qt4, macosx) and non-interactive hardcopy backends to make image files (PNG, SVG, PDF). To configure your backend change the backend parameter in your `matplotlibrc` file. You can also use the IPython [magic](https://ipython.readthedocs.io/en/stable/interactive/plotting.html) to change your current backend.

In [7]:
%matplotlib qt
# %matplotlib inline

## 4. Event handling
You can [customize events](https://matplotlib.org/3.1.1/users/event_handling.html) that are triggered by the interaction with the GUI. To connect events to a function use [`mpl_connect`](https://matplotlib.org/api/backend_bases_api.html#matplotlib.backend_bases.FigureCanvasBase.mpl_connect). An [example for keypress events](https://matplotlib.org/3.1.1/gallery/event_handling/keypress_demo.html#sphx-glr-gallery-event-handling-keypress-demo-py) can be found in the Matplotlib gallery. **Note that the event handling works with a qt backend only!**

*In this example an event is triggered when the key 'c' is pressed* 

In [17]:
import matplotlib.pyplot as plt
import numpy as np

"Prepare Data"
X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)

"Define Event"
def on_c_click(event):
    if event.key == 'c':
        print("event triggered")

fig, ax = plt.subplots(1, 1)
ax.plot(X, C)
ax.plot(X, S)
fig.canvas.mpl_connect('key_press_event', on_c_click)

plt.show()

C:\Users\sklein\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backend_managers.py:58: UserWarning: Treat the new Tool classes introduced in v1.5 as experimental for now, the API will likely change in version 2.1 and perhaps the rcParam as well
  'experimental for now, the API will likely change in ' +
C:\Users\sklein\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backend_tools.py:79: UserWarning: Treat the new Tool classes introduced in v1.5 as experimental for now, the API will likely change in version 2.1, and some tools might change name
  'experimental for now, the API will likely change in ' +


event triggered
event triggered


## 5. Customizing UI Tools with ToolManager
Matplotlib includes a [Tool Manager](https://matplotlib.org/3.1.1/api/backend_tools_api.html) which can be used to add a variety of tools to your UI. This is the method that is currently used for the haiopy package to implement custom interactivity for the plots. It's still considered experimental (see warning), but offers more options than simple event handling. **Customization with the toolmanager class also works with qt backends only!**

> The tool manager still triggers a warning refering to matplotlib v1.5/2.1 when used with v3.1.1.
> A current [Issue](https://github.com/matplotlib/matplotlib/issues/15284) about this warning

There are two classes to define custom tools for your UI.
* `ToolBase` for basic stateless tools
* `ToolToggleBase` for toggleable tools with two states
* The tool is then simply added to a figure with the `add_tool` function

Another example can be found [here](https://matplotlib.org/3.1.1/gallery/user_interfaces/toolmanager_sgskip.html#sphx-glr-gallery-user-interfaces-toolmanager-sgskip-py).

*In this example a simple Tool for hiding/showing all lines of the plot based on the ToolToggleBase class is added.*

In [19]:
plt.rcParams['toolbar'] = 'toolmanager'
from matplotlib.backend_tools import ToolBase, ToolToggleBase

class HideTool(ToolToggleBase):
    '''Show all lines'''
    default_keymap = 'A'
    description = 'Show lines'
    default_toggled = True

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def enable(self, *args):
        self.set_lines_visibility(True)

    def disable(self, *args):
        self.set_lines_visibility(False)

    def set_lines_visibility(self, state):
        for ax in self.figure.get_axes():
            for line in ax.get_lines():
                    line.set_visible(state)
        self.figure.canvas.draw()
        
fig.canvas.manager.toolmanager.add_tool('Show', HideTool) # raises AttributeError when backend is not qt
fig.show()

C:\Users\sklein\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backend_managers.py:263: UserWarning: A "Tool class" with the same name already exists, not added
  warnings.warn('A "Tool class" with the same name already exists, '


event triggered
event triggered
